# Sonecule Tests

Notebook for testing the sonecules package.

In [1]:
%load_ext autoreload
%autoreload 2

## Startup

In [2]:
import sonecules as sn

In [3]:
sn.startup()

<IPython.core.display.Javascript object>

Starting sclang process... Done.
Registering OSC /return callback in sclang... Done.
Loading default sc3nb SynthDefs... Done.
Booting SuperCollider Server... Done.


In [4]:
sn.gcc()

## Data Prep

In [5]:
import pandas as pd
import numpy as np 
import seaborn as sns

In [6]:

eeg_data = np.loadtxt("../../notebooks/data/epileptic-eeg.csv", delimiter=",")
eeg_df = pd.DataFrame(eeg_data)

penguins_df = sns.load_dataset("penguins")
penguins_df = penguins_df.dropna(subset=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g", "sex"])
penguins_df = penguins_df.reset_index(drop=True)

seaice_df = sns.load_dataset("seaice")

In [7]:
sn.playback()  # perhaps gcp()

Playback(running=False)

## Buffer Synthesis

In [8]:
from sonecules.buffersyn import Audification

In [9]:
audification = Audification(data=eeg_data[:,[0,1]], sr=256)

In [10]:
audification.synth

Synth(sc3nb_playbuf_128, {'out': 0.0, 'bufnum': 128.0, 'rate': 1.0, 'loop': 0.0, 'pan': 0.0, 'amp': 0.3})

In [11]:
audification.schedule(0, {"amp": 0.4})

In [12]:
sn.playback().start()

In [13]:
sn.playback().stop()  # stops playback (triggering of the scheduled events) but started synths will keep playing til finished

In [14]:
sn.stop()  # stops playback and stops backend

In [15]:
timbralson = sn.buffersyn.TimbralSon(eeg_data[14*256:24*256], sr=256)

In [16]:
timbralson.schedule(at=2, params={"amp": 0.03})    

In [17]:
sn.playback().start()

In [18]:
sn.stop()  # stops playback and stops backend

## ParameterSonification as Handler/Callback/Setup

In [19]:
from sonecules.scoresyn import StandardContinuousPMSon, StandardDiscretePMSon

In [20]:
from sc3nb import midicps, linlin

In [21]:
penguins_df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
328,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
329,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
330,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
331,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [22]:
# adapting linlin to take dmin dmax as params - names open for discussion
dlinlin = lambda value, dmin, dmax, y1, y2: linlin(value, x1=dmin, x2=dmax, y1=y1, y2=y2)

In [23]:
scpmson = StandardContinuousPMSon("s2", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(70), "y2": midicps(50)})
}

sn.reset() # needed for now as sonecule.reset() does not work yet
scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 


In [24]:
sn.playback().start()

In [25]:
# test_mapping = {"onset": (col, fun, mkwargs)}  

# ditched conversion for now

scpmson = StandardDiscretePMSon("s1", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)  # bounds are in pre conversion unit - but this is done differently here in the code

sn.reset() # needed for now as sonecule.reset() does not work yet
scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 


In [26]:
sn.playback().start()

In [27]:
sn.playback().time = 1.5

In [28]:
sn.stop()

## Sonification as Handler/Callback/Setup

In [29]:
from sonecules.triggersyn import DataSonogram

In [30]:
%matplotlib qt

In [32]:
sn.reset()
sn.gcc().enable_realtime()

Playback(time=0.00033855438232421875, rate=1)

In [34]:
dsg1 = DataSonogram(penguins_df, x="flipper_length_mm", y="body_mass_g", label="species")